In [1]:
# Step 1: Install required libraries
!pip install -q langchain langchain-community langchain-google-genai google-generativeai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.2 MB/s eta 0:00:00


In [ ]:
##Example of Pydantic
# from pydantic import BaseModel, ValidationError

# class User(BaseModel):
#   name: str
#   age: int
#   email: str

# try:
#   user = User(name="Alice", age=30, email="alice@gmail.com")
# except ValidationError as e:
#   print(e)
# print(user)

name='Alice' age=30 email='alice@gmail.com'


In [2]:
# Step 2: Import necessary modules
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate
)
import json
from langchain_core.output_parsers.pydantic import PydanticOutputParser
import os
import getpass

In [3]:
# Step 2: Creating Customized Output Parser
class LanguageOutputStructure(BaseModel):
  original_sentence: str = Field(description="Sentence asked by user")
  desired_language: str = Field(description="Desired language in which sentence to be translated")
  translated_sentence: str = Field(description="Translated sentence for a given sentence in given language")

In [4]:
parser = PydanticOutputParser(pydantic_object=LanguageOutputStructure)

In [5]:
system_prompt_template = PromptTemplate(
    template = """\n{formatted_instructions}\n You are a language translator. An English speaker wants to translate
    {original_sentence} to {desired_language}. Tell him the correct answer.""",
    input_variables=["original_sentence", "desired_language"],
    partial_variables={"formatted_instructions": parser.get_format_instructions()}
)

system_message_template = SystemMessagePromptTemplate(prompt=system_prompt_template)

In [6]:
user_prompt_template = PromptTemplate(
    template = "Translate {original_sentence} to {desired_language}",
    input_variables=["original_sentence", "desired_language"]
)

user_message_template = HumanMessagePromptTemplate(prompt=user_prompt_template)

In [7]:
# Step 4: Set up your Google API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [8]:
# Step 5: Initialize the Gemini model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")  # Use 'gemini-pro' as the model

In [9]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    system_message_template,
    user_message_template
])

In [10]:
messages = chat_prompt_template.format_messages(original_sentence="I love pizza!", desired_language="French")

In [11]:
chat_response_model = chat_model.invoke(messages)

In [12]:
formatted_response = parser.invoke(chat_response_model)

In [13]:
print(f"Answer Type: {type(formatted_response)}")
print(formatted_response)

Answer Type: <class '__main__.LanguageOutputStructure'>
original_sentence='I love pizza!' desired_language='French' translated_sentence="J'adore la pizza\xa0!"


In [14]:
#Convert the formatted response to JSON for better readability
json_output = formatted_response.json()
output_dict = json.loads(json_output)

<ipython-input-14-3c537226a7e1>:2: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json_output = formatted_response.json()


In [15]:
print(f"Answer: {output_dict}")

Answer: {'original_sentence': 'I love pizza!', 'desired_language': 'French', 'translated_sentence': "J'adore la pizza\xa0!"}
